<a href="https://colab.research.google.com/github/gongboogi/SchoolWork/blob/main/HonGong-mldl/hg_mldl_chapter05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**hg_mldl chapter 05-2**
교차 검증과 그리드 서치

## 검증 세트

테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다.

테스트 세트를 사용하지 않고 측정하는 방법 -> **훈련세트를 나눈다**

이 데이터를 **검증 세트**라고 부름


In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine-date')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# 훈련세트와 테스트 세트 나누기
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42) 

In [ ]:
# 훈련세트를 다시 나누기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42) # train_input의 약 20%를 val_input(검증세트)로 만듦

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

DecisionTreeClassifier(random_state=42)

In [ ]:
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))


0.9971133028626413
0.864423076923077


훈련세트에 과대적합 되었다.

## 교차검증

검증 세트를 떼어 내어 평가한느 과정을 여러 번 반복하고

평균을 내어 최종 검증 점수를 얻는다.
<br>
<br>
3-폴드 교차 검증: 훈련 세트를 세 부분으로 나눠서 교차 검증을 수행하는 것.

훈련세트를 몇 부분으로 나누냐에 따라 다르게 부른다. (K-폴드 교차 검증)


In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01140594, 0.0103519 , 0.01068377, 0.01081038, 0.01022243]), 'score_time': array([0.00143528, 0.00127006, 0.00129676, 0.0012784 , 0.00120735]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np
print(np.mean(scores['test_score'])) # 검증 폴드 점수의 평균

0.855300214703487


In [ ]:
# 분할기 지정
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
# 훈련세트를 섞은 후 10-폴드 교차 검증을 수행
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # n_splits: 몇(k) 폴드 교차 검증을 할지 지정
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719




---

## 하이퍼파라미터 튜닝

**모델 파라미터**: 모델이 학습하는 파라미터
**하이퍼파라미터**: 모델이 학습할 수 없어서 사용자가 지정해야하는 파라미터


max_depth의 최적값은 min_samples_split 매개변수의 값이 바뀌면 함께 달라짐.
-> 두 매개변수를 동시에 바꿔가며 최적 값을 찾아야한다.



In [ ]:
# 그리드 서치

from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) #그리드 서치 객체 생성 

In [ ]:
# 결정트리 모델 min_impurity_decrease값을 바꿔가며 총 5번 실행
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
# best_estimator_: 그리드 서치로 찾은 최적의 매개변수가 저장됨
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는  mean_test_score의 키에 저장
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
# 가장 큰 값의 인덱스 추출하기
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


### **과정정리**
1. 탐색할 매개변수 지정
2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개 변수 조합 찾기. -> 그리드 서치 객체에 저장됨
3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종모델 훈련. -> 그리드 서치 객체에 저장됨

In [ ]:
# min_impurity_decrease: 노드를 분할하기 위한 불순도 감소 최소량 지정
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1), 
          'min_samples_split': range(2, 100, 10)}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
# 최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

매개변수 값의 범위나 간격을 미리 정하기 어려울 때, 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달.

**싸이파이(scipy)라이브러리**: 적분, 보간, 선형 대수, 확률 등을 포함한 수치 계산 전용 라이브러리 

In [ ]:
from scipy.stats import uniform, randint #uniform, randint: 주어진 범위에서 고르게 값을 뽑는다.(균등 분포에서 샘플링함)

In [ ]:
rgen = randint(0,10)
rgen.rvs(10)

array([7, 3, 5, 0, 1, 4, 9, 6, 9, 9])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 96,  99,  94, 112, 102,  94, 105,  87,  96, 115]))

In [ ]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.09727261, 0.99369102, 0.84084506, 0.67798415, 0.0648257 ,
       0.71960772, 0.32499693, 0.01200034, 0.54878154, 0.69752023])

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), 
                        params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7faf970aa310>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7faf9748d2d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7faf970aa790>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7faf970aa490>},
                   random_state=42)

In [ ]:
#매개변수 조합 출력
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
# 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
# 최적의 모델은 best_estimator_ 속성에 저장되어있다.
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
